In [1]:
import os
import torch
import torchvision
import numpy as np
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from skimage import io, transform
from tensorflow.examples.tutorials.mnist import input_data


In [22]:
# load image
mnist = input_data.read_data_sets('../../MNIST', one_hot=True)
z_dim = 10
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
batch_size = 128
learning_rate = 1e-3
print X_dim

Extracting ../../MNIST/train-images-idx3-ubyte.gz
Extracting ../../MNIST/train-labels-idx1-ubyte.gz
Extracting ../../MNIST/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST/t10k-labels-idx1-ubyte.gz
784


In [23]:
# define some parameter
if not os.path.exists('./results'):
    os.mkdir('./results')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


img_transform = transforms.Compose([
    transforms.ToTensor(),
    ])


In [5]:
# load auto encoder
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(X_dim, h_dim),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(h_dim, X_dim), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
# instantiate model and define optimizer
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [24]:
# train network
for it in range(10000):
    img, _ = mnist.train.next_batch(batch_size)
    img = Variable(torch.from_numpy(img)).cuda()
    # ===================forward=====================
    output = model(img)
    #print output
    loss = criterion(output, img)
    # print loss
    #input()
    # ===================backward====================
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # ===================log========================
    
    if it % batch_size == 0:
        epoch = it/batch_size
        print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
        pic = to_img(output.cpu().data)
        save_image(pic, './results/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './garment_autoencoder.pth')

epoch [1/200], loss:0.1042
epoch [2/200], loss:0.0619
epoch [3/200], loss:0.0535
epoch [4/200], loss:0.0471
epoch [5/200], loss:0.0436
epoch [6/200], loss:0.0429


KeyboardInterrupt: 